<h1 align="center"><font size="5">Applied Data Science Capstone - Nikhil M</font></h1>

This notebook contains the work for the Applied Data Science Capstone.

Loading the required libraries:

In [3]:
import numpy as np
import pandas as pd

In [4]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [5]:
import requests # library to handle requests
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from lxml import etree

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  17.97 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  28.37 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  55.00 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.56 MB/s
vincent-0.4.4- 100% |###################

In [7]:
#Initialize constants for FourSquare
CLIENT_ID = '5SVQZ3G4RIN54PVDABFRLPGLCS0Y0ZF5IAZPP4NXQEWNHYA4' # your Foursquare ID
CLIENT_SECRET = 'ZAIO0JVOYHPVMYX011CCTQ2BFHZSEHT3THLJGEXPVFLSHN40' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [36]:
#Extract neighborhoods in Toronto
base_url = "https://en.wikipedia.org/wiki/List_of_city-designated_neighbourhoods_in_Toronto"
tables = pd.read_html(base_url) 

#Extract table with list of neighborhoods
df_neighborhoods = tables[0]

#Assign column headers & drop unnecessary columns
df_neighborhoods.columns = df_neighborhoods.iloc[0]
df_neighborhoods = df_neighborhoods.reindex(df_neighborhoods.index.drop(0))
df_neighborhoods = df_neighborhoods.drop(['CDN number', 'Neighbourhoods covered' , 'Map'], axis=1)
df_neighborhoods.columns = ['Neighborhood', 'Borough']

df_neighborhoods.head()

,Neighborhood,Borough
1,Agincourt North,Scarborough
2,Agincourt South-Malvern West,Scarborough
3,Alderwood,Etobicoke
4,Annex,Old City of Toronto
5,Banbury-Don Mills,North York


In [48]:
#Get Lat & Lan for all neighborhoods
geolocator = Nominatim(user_agent="toronto_explorer")
toronto_location = geolocator.geocode("Toronto")

for index, row in df_neighborhoods.iterrows():
    location = geolocator.geocode(row["Neighborhood"]+ ' ' + row["Borough"] + ' Toronto')
    if location is not None:
        df_neighborhoods.at[index, 'Latitude'] = location.latitude
        df_neighborhoods.at[index, 'Longitude'] = location.longitude
    
df_neighborhoods.head()

,Neighborhood,Borough,Latitude,Longitude
1,Agincourt North,Scarborough,43.808038,-79.266439
2,Agincourt South-Malvern West,Scarborough,43.710434,-79.453340
3,Alderwood,Etobicoke,43.601717,-79.545232
4,Annex,Old City of Toronto,43.670338,-79.407117
5,Banbury-Don Mills,North York,43.734804,-79.357243


In [ ]:
toronto_location = geolocator.geocode("Toronto")

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[toronto_location.latitude, toronto_location.longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neighborhoods['Latitude'], df_neighborhoods['Longitude'], df_neighborhoods['Borough'], df_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    return(nearby_venues)

toronto_neighborhood_venues = getNearbyVenues(names=df_neighborhoods['Neighborhood'],
                                   latitudes=df_neighborhoods['Latitude'],
                                   longitudes=df_neighborhoods['Longitude']
                                  )

toronto_neighborhood_venues.head()

Agincourt North
Agincourt South-Malvern West
Alderwood
Annex
Banbury-Don Mills
Bathurst Manor
Bay Street Corridor
Bayview Village
Bayview Woods-Steeles
Bedford Park-Nortown
Beechborough-Greenbrook
Bendale
Birchcliffe-Cliffside
Black Creek
Blake-Jones
Briar Hill-Belgravia
Bridle Path-Sunnybrook-York Mills
Broadview North
Brookhaven-Amesbury
Cabbagetown-South St. James Town
Caledonia-Fairbank
Casa Loma
Centennial Scarborough
Church-Yonge Corridor
Clairlea-Birchmount
Clanton Park
Cliffcrest
Corso Italia-Davenport
Crescent Town
Danforth Village - East York
Danforth Village - Toronto
Don Valley Village
Dorset Park
Dovercourt-Wallace Emerson-Junction
Downsview-Roding-CFB
Dufferin Grove
East End-Danforth
Edenbridge-Humber Valley
Eglinton East
Elms-Old Rexdale
Englemount-Lawrence
Eringate-Centennial-West Deane
Etobicoke West Mall
Flemingdon Park
Forest Hill North
Forest Hill South
Glenfield-Jane Heights
Greenwood-Coxwell
Guildwood
Henry Farm
High Park North
High Park-Swansea
Highland Creek
Hil

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt North,43.808038,-79.266439,Saravanaa Bhavan South Indian Restaurant,43.810117,-79.269275,Indian Restaurant
1,Agincourt North,43.808038,-79.266439,Booster Juice,43.809915,-79.269382,Juice Bar
2,Agincourt North,43.808038,-79.266439,Dollarama,43.808889,-79.268889,Discount Store
3,Agincourt North,43.808038,-79.266439,Shoppers Drug Mart,43.808119,-79.269744,Pharmacy
4,Agincourt North,43.808038,-79.266439,The Beer Store,43.809286,-79.263676,Beer Store


In [81]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [82]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [83]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Bakery,Ice Cream Shop,Chinese Restaurant,Fried Chicken Joint,Bank,Japanese Restaurant,Movie Theater,Taco Place,Beer Store,Pharmacy
1,Agincourt South-Malvern West,Pizza Place,Italian Restaurant,Fast Food Restaurant,Construction & Landscaping,Fish Market,Bookstore,Latin American Restaurant,Furniture / Home Store,Mediterranean Restaurant,Pharmacy
2,Alderwood,Pizza Place,Pharmacy,Coffee Shop,Skating Rink,Pub,Pool,Sandwich Place,Gym,Elementary School,Discount Store
3,Annex,Pizza Place,Thai Restaurant,Bookstore,Ice Cream Shop,Indian Restaurant,Burrito Place,Burger Joint,Middle Eastern Restaurant,Bistro,Korean Restaurant
4,Banbury-Don Mills,Park,Coffee Shop,Intersection,Yoga Studio,Dongbei Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [86]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Agincourt North,Scarborough,43.808038,-79.266439,4,Bakery,Ice Cream Shop,Chinese Restaurant,Fried Chicken Joint,Bank,Japanese Restaurant,Movie Theater,Taco Place,Beer Store,Pharmacy
2,Agincourt South-Malvern West,Scarborough,43.710434,-79.453340,1,Pizza Place,Italian Restaurant,Fast Food Restaurant,Construction & Landscaping,Fish Market,Bookstore,Latin American Restaurant,Furniture / Home Store,Mediterranean Restaurant,Pharmacy
3,Alderwood,Etobicoke,43.601717,-79.545232,0,Pizza Place,Pharmacy,Coffee Shop,Skating Rink,Pub,Pool,Sandwich Place,Gym,Elementary School,Discount Store
4,Annex,Old City of Toronto,43.670338,-79.407117,4,Pizza Place,Thai Restaurant,Bookstore,Ice Cream Shop,Indian Restaurant,Burrito Place,Burger Joint,Middle Eastern Restaurant,Bistro,Korean Restaurant
5,Banbury-Don Mills,North York,43.734804,-79.357243,3,Park,Coffee Shop,Intersection,Yoga Studio,Dongbei Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [89]:
# create map
map_clusters = folium.Map(location=[toronto_location.latitude, toronto_location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters